In [0]:
import numpy as np
import pandas as pd 
import re

# import nltk
# !pip install xlrd
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()
# from textblob import TextBlob
# from spellchecker import SpellChecker
# spell = SpellChecker()

# from fuzzywuzzy import process
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import scipy.sparse as sp
from scipy.sparse import csr_matrix

In [0]:
 %pip install openpyxl

In [0]:
path1 = "uncleaneddata1.csv"
path2 = "cleanedfile1.csv" 

# Read the files from the corrected paths
vs = pd.read_csv(path1, index_col=None)
mad = pd.read_csv(path2,index_col=None)

In [0]:
# To refer back to original , as it takes long to load original data 
madcopy=mad.copy()
vscopy=vs.copy()

In [0]:
##EDA ,Feature Engineering and Feature Selection ##


In [0]:
mad.columns,mad.shape

In [0]:
vs.columns,vs.shape

In [0]:
# mad=madcopy.copy()
# mad['ProvinceID'].value_counts()

In [0]:
# mad = mad[['AddressID', 'StreetNumber',
#         'StreetAddress1', 'StreetAddress2', 'UnitNumber', 'City', 'PostalCode','ProvinceID']].rename(columns={'PostalCode': 'Postal Code'})

In [0]:
# ProvinveIddata=pd.read_excel('ProvinceId.xlsx')

In [0]:
# mad = pd.merge(mad, ProvinveIddata, on='ProvinceID', how='left')

In [0]:
# mad['Province'].nunique()

In [0]:
# mad.drop('ProvinceID', axis=1, inplace=True)
mad.head(2)

In [0]:
# def get_address(df):
#     df['Address'] = df.iloc[:,1:].apply(lambda x: ' '.join(str(value) for value in x if pd.notna(value)), axis=1)
#     return df

In [0]:
# vs=get_address(vs)
# mad=get_address(mad)

In [0]:
# mad.reset_index(drop=True,inplace=True)
# vs.reset_index(drop=True,inplace=True)

In [0]:
#Save work so far for multiple usages 
# excel_filename = 'vs_address.xlsx'
# vs.to_excel(excel_filename,index=False)
# excel_filename = 'time_address.xlsx'
# mad.to_excel(excel_filename,index=False)

In [0]:
# vs_address=pd.read_excel('vs_address.xlsx',index_col=None )
# mad_address=pd.read_excel('time_address.xlsx',index_col=None)

In [0]:
vs_address=vs.copy()
mad_address=mad.copy()

In [0]:
mad_address.head(2)

In [0]:
# type(mad_address.iloc[0,7])
# len(mad_address.iloc[0,7])

In [0]:
vs_address.head(2)

In [0]:
def data_preprocessing(df):  
    df['clean_Address'] = df['finaladd'].apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', str(x)))
    df['clean_Address'] = df['clean_Address'].str.lower()    
    return df   
    
    # review = review.split()
    # review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    # review = ' '.join(review)
    # corpus.append(review)


    # misspelled = spell.unknown(str)
    # print(misspelled)
    # new_doc = TextBlob(updated_docx)
    # result = new_doc.correct()
    # print(result)

In [0]:
vs=data_preprocessing(vs_address)
mad=data_preprocessing(mad_address)


In [0]:
# count_vectorizer = CountVectorizer(binary=True)
# vs_sparse=csr_matrix(count_vectorizer.fit_transform(vs['clean_Address']).astype(np.float32))

# vs_count_matrix = count_vectorizer.fit_transform(vs_address['clean_Address'])
# vs_count_df = pd.DataFrame(vs_count_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())
# vs_count_df.head(2)

In [0]:
# count_vectorizer.vocabulary_

In [0]:
# count_vectorizer.get_feature_names_out().shape

In [0]:
# type(vs_count_df.iloc[0,0])

In [0]:
mad.head()

In [0]:
vs.isnull().sum(axis = 0)

In [0]:
mad.isnull().sum(axis = 0)

In [0]:
# count_vectorizer = CountVectorizer(sparse_output=True)
# count_vectorizer = CountVectorizer(dtype='numpy.int32')

# count_vectorizer = CountVectorizer(binary=True)
# sparse=csr_matrix(count_vectorizer.fit_transform(mad['clean_Address']).astype(np.float32))


# count_vectorizer.fit_transform(mad['clean_Address']).toarray().shape

In [0]:
# mad_sparse = np.array(sparse)
# np.save('mad_sparse.npy', mad_sparse)
# mad_sparse = np.load('mad_sparse.npy')

In [0]:
# vs1=vs.iloc[7000:,:].reset_index(drop=True) 
clean_addresses_df1 = vs['clean_Address'].tolist()
clean_addresses_df2 = mad['clean_Address'].tolist()
combined_clean_addresses = clean_addresses_df1 + clean_addresses_df2

count_vectorizer = CountVectorizer(binary=True)
count_matrix_combined=csr_matrix(count_vectorizer.fit_transform(combined_clean_addresses).astype(np.float32))



In [0]:
count_vectorizer.get_feature_names_out().shape

In [0]:
vs_sparse=csr_matrix(count_vectorizer.transform(vs['clean_Address']).astype(np.float32))
# vs_sparse_df = pd.DataFrame(vs_sparse)

In [0]:
mad_sparse=csr_matrix(count_vectorizer.transform(mad['clean_Address']).astype(np.float32))

In [0]:
vs_sparse.shape

In [0]:
cosine_similarities = cosine_similarity(vs_sparse,mad_sparse)

In [0]:
cosine_similarities.shape

In [0]:
type(cosine_similarities)

In [0]:
similarity_df = pd.DataFrame(cosine_similarities, columns=mad.index)

In [0]:
similarity_df.tail(2)

In [0]:
mad.head(10)

In [0]:
vs.head(10)

In [0]:

top_3_addresses = similarity_df.apply(lambda x: mad['clean_Address'].iloc[x.argsort()[-3:]].tolist(), axis=1)
top_3_addressesIDs = similarity_df.apply(lambda x: mad['Cleaned_Add_id'].iloc[x.argsort()[-3:]].tolist(), axis=1)
top_3_values = similarity_df.apply(lambda x: list(round(val, 2) for val in x.iloc[x.argsort()[-3:]]), axis=1)

vs['top_3_matches'] = top_3_addresses
vs['top_3_addressesIDs'] = top_3_addressesIDs
vs['top_3_values'] = top_3_values


In [0]:
match_data= vs[['Uncleaned_add_id','clean_Address','top_3_addressesIDs', 'top_3_matches','top_3_values']]
match_data.head(2)

In [0]:
# from collections import Counter
# for i in top_3_addresses:    
#     print(Counter(i).values())

In [0]:
match_data.isnull().sum(axis = 0)

In [0]:
match_data.shape

In [0]:
# match_data.to_excel('RestofCanada.xlsx',index=False)

In [0]:
# import ast

In [0]:
# match_data['top_3_addressesIDs'] = match_data['top_3_addressesIDs'].apply(ast.literal_eval)
# match_data['top_3_values'] = match_data['top_3_values'].apply(ast.literal_eval)


In [0]:
match_data.iloc[0,4]

In [0]:
type(match_data.iloc[0,4][2])

In [0]:
len(match_data.iloc[0,4])

In [0]:
# nlm=match_data.iloc[0,2]
# for i in nlm:
#     print(i)

In [0]:
# import math

# hello = []
# for i in match_data['top_3_values']:
#     if isinstance(i, (int, float)):
#         if not math.isnan(i):
#             hello.append(int(i))
#     elif isinstance(i, list):
#         for j in i:
#             if not math.isnan(j):
#                 hello.append(int(j))
#     else:
#         # Handle other cases or raise an exception if needed
#         pass



In [0]:
# match_data12=match_data.iloc[:5,:]
# addressesIDs12 =  [j for i in match_data12['top_3_addressesIDs'] if isinstance(i, list) for j in i]
# VS_Time_Ids12= pd.DataFrame({'FCTRefNum': match_data12['FCTRefNum'].repeat(3),
# 'addressesIDs12' : [j for i in match_data12['top_3_addressesIDs'] if isinstance(i, list) for j in i]
# })
# len(addressesIDs12)

In [0]:
match_data.tail(2)

In [0]:
VS_Time_Ids= pd.DataFrame({'Uncleaned_add_id': match_data['Uncleaned_add_id'].repeat(3),                           
'addressesIDs':[j for i in match_data['top_3_addressesIDs'] if isinstance(i, list) for j in i]
,'match_score': [j for i in match_data['top_3_values'] if isinstance(i, list) for j in i]
# 'flattened_list' : [j for i in match_data['top_3_values'] for j in (i if isinstance(i, list) else [i])]
                          })

In [0]:
VS_Time_Ids.head(3)

In [0]:
VS_Time_Ids.to_csv('Matching_Ids.csv',index=False)